In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import torch.cuda

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Veri yolu - Klasördeki verilerin kategori alt klasörlerde bulunduğunu varsayalım.
data_dir = 'data/'

# Veri dönüşümleri (örnek olarak)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Görselleri istediğiniz boyuta dönüştürebilirsiniz
    transforms.ToTensor(),          # Görselleri tensöre dönüştür
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizasyon (örnek)
])

# Veri yükleyici parametreleri
batch_size = 32
num_workers = 4

# Veriyi yükleyin
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


Using device: cuda:0


In [2]:
num_classes = len(dataset.classes)
num_classes

39

In [9]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Giriş boyutunu uygun şekilde ayarlayın
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Modeli oluştur
num_classes = len(dataset.classes)  # Sınıf sayısı otomatik olarak belirlenir
model = CNNModel(num_classes).to(device)


In [10]:
# Eğitim parametreleri
learning_rate = 0.001
num_epochs = 10

# Kayıp fonksiyonu ve optimizasyon fonksiyonu
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
correct = 0
total = 0

for epoch in range(num_epochs):
    for images, labels in dataloader:
        # Veriyi GPU üzerine taşı
        images, labels = images.to(device), labels.to(device)

        # İleri geçiş
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Geriye doğru yayılım ve parametre güncelleme
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()} Accuracy: {accuracy}')


Epoch [1/10] Loss: 0.4729686677455902 Accuracy: 72.86105901024384
Epoch [2/10] Loss: 0.34890151023864746 Accuracy: 80.5132736978791
Epoch [3/10] Loss: 0.15320201218128204 Accuracy: 84.88133025537441
Epoch [4/10] Loss: 0.3196655809879303 Accuracy: 87.78314817486654
Epoch [5/10] Loss: 0.06553100794553757 Accuracy: 89.73416534410619
Epoch [6/10] Loss: 0.21722841262817383 Accuracy: 91.07301736161209
Epoch [7/10] Loss: 0.02120833285152912 Accuracy: 92.11281612630624
Epoch [8/10] Loss: 0.2340955138206482 Accuracy: 92.92309911989612
Epoch [9/10] Loss: 0.00041523922118358314 Accuracy: 93.57255646932461
Epoch [10/10] Loss: 0.0008148965425789356 Accuracy: 94.08021930457366


In [11]:
# Modeli kaydet
torch.save(model.state_dict(), 'plant.pth')

In [12]:
# Model yapısını oluştur
model = CNNModel(num_classes)  # Model yapısını aynı şekilde oluşturmalısınız

# Kaydedilmiş parametreleri yükle
model.load_state_dict(torch.load('plant.pth'))

# Modeli tahminlerde kullanabilirsiniz
model.eval()  # Modeli tahmin modunda etkinleştirin
# Burada tahminlerinizi yapabilirsiniz


CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=39, bias=True)
)

In [13]:
import cv2
from PIL import Image

In [14]:
image_path = '/home/berkkucukk/PythonProjects/Plant/data/Apple___Apple_scab/image (1).JPG'  # Test görüntüsünün dosya yolu
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Görüntüyü BGR'den RGB'ye dönüştür
image_pil = Image.fromarray(image)

In [15]:
# Giriş görüntüsünü modele uygun hale getir
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_image = transform(image_pil)
input_image = input_image.unsqueeze(0)  # Batch boyutunu ekleyin (1 görüntü için)

In [16]:
with torch.no_grad():
    output = model(input_image)

# Sınıflandırma sonuçlarını al
_, predicted = torch.max(output, 1)

# Sınıf tahminini bul
class_index = predicted.item()
class_label = dataset.classes[class_index]  # 'dataset' veri kümesinin sınıflarını içerdiğini varsayalım

print(f'Tahmin edilen sınıf: {class_label}')

Tahmin edilen sınıf: Apple___Apple_scab
